# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


# Data Preprocessing
Every data, that we wish to fed into a learning algorithm to learns the patterns and trends in it, must be frist cleaned and made ready for the algorithm to understand it.

In [2]:
#loading dataset

columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean",
           "concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se",
           "perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se",
           "fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst",
           "compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst",]
df = pd.read_csv("wdbc.csv",names = columns)
print(df.shape)
df.head(10)

(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [3]:
# droping id columns because it is not needed.
del df['id']
#checking for column with a empty cell i.e containing NA value
df.isna().any()

diagnosis                  False
radius_mean                False
texture_mean               False
perimeter_mean             False
area_mean                  False
smoothness_mean            False
compactness_mean           False
concavity_mean             False
concave points_mean        False
symmetry_mean              False
fractal_dimension_mean     False
radius_se                  False
texture_se                 False
perimeter_se               False
area_se                    False
smoothness_se              False
compactness_se             False
concavity_se               False
concave points_se          False
symmetry_se                False
fractal_dimension_se       False
radius_worst               False
texture_worst              False
perimeter_worst            False
area_worst                 False
smoothness_worst           False
compactness_worst          False
concavity_worst            False
concave points_worst       False
symmetry_worst             False
fractal_di

In [4]:
# checking total malignant(cancer) cells and benign(non-cancer) cells.
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [5]:
# changing categorical data into int numbers so that learning algorithms can understand them.
# the malignant representation "M" will be converted to 1 and benign representation to 0.
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df["diagnosis"].values)
df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [6]:
# now we will be spliting dataset into training and testing parts.
y = df.iloc[:,0].values
x = df.iloc[:,0:].values
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
X_train.shape

(426, 31)

In [7]:
# now we will feature scale the data. It is important so that all the valuse will have same weight for
# our learning algorithm.

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Model Building
Now that our data is ready to be fed in learning algorithms, we will proceed toward model building. We will be using keras to 
make a binary classifier.But before making model let's first see what accuracy we can get out of it using sklearn's stratified k-fold cross validation.

Stratified k-fold cross validation.is a resampling technique that will provide an estimate of the performance of the model. It does this by splitting the data into k-parts, training the model on all parts except one which is held out as a test set to evaluate the performance of the model. This process is repeated k-times and the average score across all constructed models is used as a robust estimate of performance.

In [8]:
# define the keras model
def buildModel():
    model = Sequential()
    model.add(Dense(31, input_dim=31, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
# evaluate model with standardized dataset
# the KerasClassifier allows to use sklearn with it. It takes a model
estimator = KerasClassifier(build_fn=buildModel, epochs=200, batch_size=50, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
# print avg accuracy along with std.
print("Avg Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Avg Accuracy: 99.77% (0.70%)


So we see that avg accuracy would be arround 99.77%. Now lets see what the final accuracy will be.

In [10]:
model = buildModel()
# fit the keras model on the dataset
model.fit(X_train, Y_train, epochs=200, batch_size=50)

Epoch 1/200
426/426 [==============================] - 0s 255us/step - loss: 0.7449 - accuracy: 0.4108
Epoch 2/200
426/426 [==============================] - 0s 42us/step - loss: 0.5807 - accuracy: 0.6526
Epoch 3/200
426/426 [==============================] - 0s 49us/step - loss: 0.4621 - accuracy: 0.8756
Epoch 4/200
426/426 [==============================] - 0s 44us/step - loss: 0.3705 - accuracy: 0.9484
Epoch 5/200
426/426 [==============================] - 0s 44us/step - loss: 0.2976 - accuracy: 0.9601
Epoch 6/200
426/426 [==============================] - 0s 54us/step - loss: 0.2401 - accuracy: 0.9742
Epoch 7/200
426/426 [==============================] - 0s 59us/step - loss: 0.1921 - accuracy: 0.9789
Epoch 8/200
426/426 [==============================] - 0s 54us/step - loss: 0.1565 - accuracy: 0.9836
Epoch 9/200
426/426 [==============================] - 0s 59us/step - loss: 0.1285 - accuracy: 0.9859
Epoch 10/200
426/426 [==============================] - 0s 56us/step - loss: 0.10

426/426 [==============================] - 0s 47us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 81/200
426/426 [==============================] - 0s 43us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 82/200
426/426 [==============================] - 0s 49us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 83/200
426/426 [==============================] - 0s 45us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 84/200
426/426 [==============================] - 0s 49us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 85/200
426/426 [==============================] - 0s 49us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 86/200
426/426 [==============================] - 0s 44us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 87/200
426/426 [==============================] - 0s 42us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 88/200
426/426 [==============================] - 0s 42us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 89/200
426/426 [==============================] - 0s 42us/step - loss: 0.0012 - 

426/426 [==============================] - 0s 47us/step - loss: 3.0524e-04 - accuracy: 1.0000
Epoch 157/200
426/426 [==============================] - 0s 47us/step - loss: 3.0094e-04 - accuracy: 1.0000
Epoch 158/200
426/426 [==============================] - 0s 49us/step - loss: 2.9674e-04 - accuracy: 1.0000
Epoch 159/200
426/426 [==============================] - 0s 46us/step - loss: 2.9190e-04 - accuracy: 1.0000
Epoch 160/200
426/426 [==============================] - 0s 48us/step - loss: 2.8794e-04 - accuracy: 1.0000
Epoch 161/200
426/426 [==============================] - 0s 49us/step - loss: 2.8326e-04 - accuracy: 1.0000
Epoch 162/200
426/426 [==============================] - 0s 49us/step - loss: 2.7925e-04 - accuracy: 1.0000
Epoch 163/200
426/426 [==============================] - 0s 45us/step - loss: 2.7544e-04 - accuracy: 1.0000
Epoch 164/200
426/426 [==============================] - 0s 42us/step - loss: 2.7082e-04 - accuracy: 1.0000
Epoch 165/200
426/426 [===================

In [11]:
# evaluate the keras model on train set.
_, accuracy = model.evaluate(X_train, Y_train)
print('Accuracy: %.2f' % (accuracy*100))

426/426 [==============================] - 0s 79us/step
Accuracy: 100.00


In [12]:
# evaluating model on test set.
_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

143/143 [==============================] - 0s 112us/step
Accuracy: 99.30


In [13]:
y_test_pred = model.predict(X_test)
y_test_pred = [int(round(x[0],0)) for x in y_test_pred]

print("Predicted test values:\n", y_test_pred)
print("Original test values:\n", Y_test)

Predicted test values:
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Original test values:
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1
 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0
 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1]


In [14]:
# now we will use confusion matrix to see how many times the models predicted accurately.
# the count of rightly predicted will be on the main diagnol of the matrix.
# the count of wrongly predicted will be on the other diagnol of the matrix.
cm = confusion_matrix(Y_test, y_test_pred)
print(cm)
print('Testing Accuracy = {}%'.format(  (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][1] + cm[1][0] + cm[0][1])))

[[90  0]
 [ 1 52]]
Testing Accuracy = 0.993006993006993%
